In [ ]:
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
# import datasets.omniglot as om            
import random
import matplotlib.pyplot as plt
train_transform = transforms.Compose(
    [transforms.Resize((84, 84)),
     transforms.ToTensor()])


# ds_train = om.Omniglot("../data/omni/", background=True, download=True, train=True, transform=train_transform, all=all)

In [ ]:
ds_train = torchvision.datasets.Omniglot(
    "../data/omni/", background=True, transform=
    transforms.Compose([
        transforms.Resize((28,28)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.92206*256], std=[0.08426*256*256]),
        
    ])
)


In [ ]:
# ds_train._flat_character_images

In [ ]:
ds_train._flat_character_images

In [ ]:
from collections import defaultdict
def alphabet_to_list_instances(ds):
    res = defaultdict(list)
    for i, (img_path, target) in enumerate(ds._flat_character_images):
        alphabet = ds_train._characters[target].split('/', maxsplit=1)[0]
        res[alphabet].append(i)
        
    return res
alphabet_to_indices = alphabet_to_list_instances(ds_train)

In [ ]:
list(alphabet_to_indices.keys())

In [ ]:
from torch.utils.data import Sampler
from typing import List, Dict
    
class ContinuousSequenceSampler(Sampler):
    r"""
    Samples elements in sequences of random length. Each sequence contains elements from one class.
    """

    def __init__(self, class_to_indices: Dict, min_seq_len, max_seq_len, seed=None):
        self.rs = random.Random(seed)
        self.min_len, self.max_len = min_seq_len, max_seq_len
        self.iterators = {
            cl: iter(self.rs.sample(indices, k=len(indices))) # permutation, sample without replacement
            for cl, indices in class_to_indices.items()
        }
        self.non_empty_classes = list(self.iterators.keys())
        total_len = sum([len(indices) for indices in class_to_indices.values()])
        self._len = total_len

    def __iter__(self):
        while self.non_empty_classes:
            cl = self.rs.choice(self.non_empty_classes)
            seq_len = self.rs.randint(self.min_len, self.max_len)
            it = self.iterators[cl]
            for i in range(seq_len):
                try:
                    yield next(it)
                except StopIteration:
                    self.non_empty_classes.remove(cl)
                    break

    def __len__(self):
        return self._len

In [ ]:
d = 'abcdef'
for seed in range(1000):
    sampler = ContinuousSequenceSampler({1:[0,1,5], 2:[2,3], 3:[4]}, 2, 3, seed=seed)
    assert sorted(iter(sampler)) == sorted(range(6))
#     == 'abcdef'
# list(sampler)

In [ ]:
sampler = ContinuousSequenceSampler({1:[0,1,5], 2:[2,3,4]}, 3, 3, seed=None)
list(sampler)

In [ ]:
sampler_train = ContinuousSequenceSampler(alphabet_to_indices, min_seq_len=3, max_seq_len=20, seed=0)
# list(sampler_train)

In [ ]:
dl_train = DataLoader(dataset=ds_train, sampler=sampler_train, batch_size=20)
i_train = iter(dl_train)

In [ ]:
# dl_train = DataLoader(dataset=om_t, sampler=sampler_train, batch_size=20)
# i_train = iter(dl_train)


In [ ]:
def make_grid(b, **kwargs):
    mi, ma = b.min(), b.max()
    d = ma - mi
    if d == 0:
        d = 1.
    b = (b - mi ) / d
    grid_img = torchvision.utils.make_grid(b, **kwargs)
    return grid_img 

b = next(i_train)
grid_img = make_grid(b[0], nrow=5)
plt.imshow(grid_img.permute(1, 2, 0))